### 📐 Custom Order Parameters: `q` and `p`

In this project, we introduce two **problem-specific order parameters** — `q` and `p` — designed to summarize the global state of signed networks during Monte Carlo simulations.

These parameters are rooted in concepts from statistical physics and network science, and were specifically developed for analyzing **structural balance** and **interaction patterns** in systems like social or brain networks.

---

#### 🔹 `p`: Mean Link Strength (Interaction Order Parameter)

`p` is a statistical measure of the **average signed interaction strength** in the network.

It is computed as the ratio of the sum of signed links to the sum of their absolute values:

\[
p = \frac{\sum_{i < j} A_{ij}}{\sum_{i < j} |A_{ij}|}
\]

Where \( A_{ij} \) is the weight of the link between nodes \( i \) and \( j \).

- A value close to +1 indicates a highly **cooperative** or **correlated** network.
- A value close to -1 indicates strong **conflict** or **anti-correlation**.
- It serves as an **order parameter** to detect changes in the average polarity of interactions as the system evolves.

---

#### 🔹 `q`: Two-Star Motif Alignment (Structural Order Parameter)

`q` is a structural parameter based on **two-star motifs** — configurations where two links share a common node (i.e., `j → i → k`).

It is calculated as the normalized sum of the product of link pairs forming two-step paths:

\[
q = \frac{\sum_{j \neq k} A_{ji} \cdot A_{ik}}{\sum_{j \neq k} |A_{ji} \cdot A_{ik}|}
\]

- High positive values of `q` reflect **alignment** or **structural coherence** across indirect paths.
- Values near zero suggest **randomness** or **imbalance** in the network.
- This metric is especially useful for capturing **indirect conflicts** or **frustrated loops**, which are critical in understanding complex systems like brain networks or social dynamics.

---

Both `q` and `p` act as **order parameters**, providing insight into the macroscopic state of the system and how it transitions across temperatures or conditions in the Monte Carlo simulation.

These metrics are designed specifically for this project and are meaningful in the context of **signed networks**, **structural balance theory**, and **higher-order network interactions**.


In [ ]:
import numpy as np
from tqdm import tqdm

def balance_model(Network, num_ensembles, list_edges):
    """
    Monte Carlo Metropolis algorithm to simulate structural balance in signed networks.

    Parameters:
    -----------
    Network : list of np.ndarray
        List of signed adjacency matrices (one per network/subject).
    num_ensembles : int
        Number of Monte Carlo repetitions per temperature.
    list_edges : list of lists
        Each sublist contains edges (pairs [i, j]) for the corresponding network.

    Returns:
    --------
    configuration : list
        Final network states for each temperature.
    Temp : list
        Temperatures used in the simulation.
    E_mean : np.ndarray
        Mean energy values averaged over ensembles and networks.
    q_mean : np.ndarray
        Mean q values (order parameter for two-star motifs).
    p_mean : np.ndarray
        Mean p values (order parameter for mean link strength).
    """

    E = []            # To store energies per temperature
    q = []            # To store q order parameter per temperature
    p = []            # To store p order parameter per temperature
    Temp = []         # List of temperatures
    configuration = []  # Final configurations at each temperature

    # Loop over temperature steps (e.g., 0,1,2,3)
    for T in tqdm(np.arange(0, 4, 1), desc='Temperature Loop'):
        E_ens = []  # Energies for all ensembles at this temperature
        q_ens = []  # q values for all ensembles at this temperature
        p_ens = []  # p values for all ensembles at this temperature

        # Repeat simulation num_ensembles times per temperature for averaging
        for _ in range(num_ensembles):
            # Deep copy all networks to avoid modifying originals
            network_copies = [np.copy(matrix) for matrix in Network]

            # Update each network individually
            for idx, net in enumerate(network_copies):
                edges = list_edges[idx]
                for (row, col) in edges:
                    s = 0
                    # Calculate sum over neighbors for delta_E calculation
                    for k in range(len(net)):
                        s += net[row, k] * net[k, col]

                    delta_E = 2 * s * net[row, col]

                    # Metropolis acceptance probability
                    z = np.random.random()
                    if z < min(1, np.exp(-delta_E / T)) if T != 0 else 0:
                        # Flip edge sign symmetrically
                        net[row, col] *= -1
                        net[col, row] *= -1

                # After all edges updated, compute order parameters
                E_ens.append(Energy(net))
                q_ens.append(q_func(net))
                p_ens.append(mean_of_links(net))

        # Save results for this temperature
        Temp.append(T)
        configuration.append(network_copies)

        # Reshape and average over ensembles and networks
        E_array = np.array(E_ens).reshape(num_ensembles, len(Network))
        q_array = np.array(q_ens).reshape(num_ensembles, len(Network))
        p_array = np.array(p_ens).reshape(num_ensembles, len(Network))

        E.append(np.mean(E_array, axis=0))
        q.append(np.mean(q_array, axis=0))
        p.append(np.mean(p_array, axis=0))

    # Final averages across all networks (axis=1 averages networks)
    return configuration, Temp, np.mean(E, axis=1), np.mean(q, axis=1), np.mean(p, axis=1)
